# Linear Mixed Model (LMM)
# considering that home office rate is continuous (e.g., % from 0 to 100) which is from what I understand
# If it is t's a proportion from counts (e.g., #people working from home / total surveyed) GLMM with binomial family, using successes and failures...
# I'll go with lmm for now 

In [3]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# HO rate∼cluster∗gender∗children+(1∣region)+(1∣year) => this is basically the formular 

In [18]:
df = pd.read_csv("all_home_office_clean.csv")

print(df.head())
print(df.info())


    region gender age_group children  cluster  year  home_office_percent  \
0  Belgium  total     18-24    total        1  2015                  8.9   
1  Czechia  total     18-24    total        2  2015                  2.7   
2  Denmark  total     18-24    total        1  2015                  5.8   
3  Germany  total     18-24    total        0  2015                  2.9   
4  Estonia  total     18-24    total        0  2015                  6.7   

  cluster_label  
0       High-HO  
1        Low-HO  
2       High-HO  
3        Mid-HO  
4        Mid-HO  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25030 entries, 0 to 25029
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   region               25030 non-null  object 
 1   gender               25030 non-null  object 
 2   age_group            25030 non-null  object 
 3   children             25030 non-null  object 
 4   cluster              2

In [19]:
# Check categories
print(df['cluster_label'].value_counts())
print(df['gender'].value_counts())
print(df['children'].value_counts())



cluster_label
Low-HO     11500
Mid-HO      8010
High-HO     5520
Name: count, dtype: int64
gender
total     8630
male      8270
female    8130
Name: count, dtype: int64
children
total    5880
0        5830
1        4990
2        4540
3+       3790
Name: count, dtype: int64


In [20]:
# Drop rows where gender and childreen are total (it messes up the analysis)
df = df[(df['gender'] != 'total') & (df['children'] != 'total')]

# reset index after filtering
df = df.reset_index(drop=True)


In [21]:
# Summary stats
print(df.groupby(['cluster_label', 'gender', 'children'])['home_office_percent'].describe())


                               count       mean        std   min     25%  \
cluster_label gender children                                              
High-HO       female 0         420.0  31.280476  10.774747   5.4  23.000   
                     1         350.0  35.824857   9.602291   4.3  29.725   
                     2         310.0  42.402581   9.220660  25.2  34.625   
                     3+        300.0  40.334000   8.536519  23.0  34.175   
              male   0         420.0  31.817857  10.157419   6.2  27.400   
                     1         360.0  38.112500   9.187419   4.6  32.900   
                     2         350.0  44.005143  10.026079   5.0  39.700   
                     3+        310.0  42.723871   8.689413  25.0  35.900   
Low-HO        female 0         890.0   8.669101   4.761143   0.6   4.800   
                     1         750.0   8.525200   4.590961   0.8   5.100   
                     2         680.0  10.036176   5.269225   1.2   5.800   
            

In [ ]:
# Convert to categories (it's like converting things to factor in R)
categorical_vars = ['cluster_label', 'gender', 'children', 'region', 'year']
for col in categorical_vars:
    df[col] = df[col].astype('category')


In [ ]:
#so again this is the fomular rate ~ cluster * gender * children + (1|region) + (1|year) 
# Mixed model with random intercept for region and year (it is more manual than lme4 in R)
# home office rates is the dependent variable

md = smf.mixedlm(
    "home_office_percent ~ cluster_label * gender * children",
    data=df,
    groups=df["region"], 
    re_formula="~1"
)
mfit = md.fit()
print(mfit.summary())



                               Mixed Linear Model Regression Results
Model:                         MixedLM            Dependent Variable:            home_office_percent
No. Observations:              12520              Method:                        REML               
No. Groups:                    29                 Scale:                         41.7111            
Min. group size:               120                Log-Likelihood:                -41178.9160        
Max. group size:               560                Converged:                     Yes                
Mean group size:               431.7                                                                
----------------------------------------------------------------------------------------------------
                                                       Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
----------------------------------------------------------------------------------------------------
Intercept             

Step 1 — Understanding the reference categories

In regression with categorical predictors, one category from each variable becomes the reference (baseline), and all other coefficients are interpreted relative to that.


Cluster: "High-HO" (since Low-HO and Mid-HO appear as contrasts)
Gender: "female" (since male appears as a contrast)
Children: "0" (since 1, 2, and 3+ appear as contrasts)

So:
Intercept (31.280) = estimated mean home office rate for
High-HO cluster, female, 0 children
(plus any random effect for a country/year if you have them)

Step 2 — Main effects

These tell us the average change from the reference when we change one factor, holding others constant:

Cluster
Low-HO: -22.847 → Low-HO countries have ~22.85 percentage points lower home office rates than High-HO countries.
Mid-HO: -10.679 → Mid-HO countries have ~10.68 percentage points lower rates than High-HO.

Gender
male: +0.537, p=0.228 → No significant difference between men and women overall (since p > 0.05).

Children
1 child: +4.34, p<0.001 → People with 1 child have ~4.34 points higher home office rates than those with 0 children.
2 children: +10.94, p<0.001 → +10.94 points compared to 0 children.
3+ children: +9.05, p<0.001 → +9.05 points compared to 0 children.

step 3 — Two-way interactions

These tell us how the effect of one variable changes depending on another variable.

Example:
Cluster[T.Low-HO] : Gender[T.male] = -1.978
→ In Low-HO countries, the male-female gap is ~2 points smaller (more negative) than in High-HO countries.

Cluster[T.Low-HO] : Children[T.2] = -9.603
→ In Low-HO countries, the "2 children vs 0 children" difference is ~9.6 points smaller than in High-HO countries.

In general:
Negative interaction → the effect is weaker in that combination than you'd expect from main effects alone.
Positive interaction → the effect is stronger.

Step 4 — Three-way interactions

Example:

Cluster[T.Mid-HO] : Gender[T.male] : Children[T.3+] = -1.489
→ In Mid-HO countries, for men with 3+ children, the deviation from the baseline (after accounting for all main and two-way effects) is about -1.49 points.

p = 0.099 → borderline non-significant.

Key takeaway

Cluster is highly significant — High-HO countries have much higher home office rates than Mid or Low.
Number of children shows a strong positive relationship — more children, higher home office rate — but this relationship is moderated by cluster.
Gender alone is not significant overall, but gender effects vary by cluster (significant interactions).
Many interaction terms are significant, suggesting the story is not just "main effects" — the relationship between children and home office rates depends on the country cluster, and to some extent gender.